# Spark Basics

이번 notebook에서는 앞으로 학습할 Spark의 활용에 대해서 예습하고 실습해보도록 하겠습니다. 
특히 이번 실습에서는 스파크에서 중요한 역할을 하는 두가지 오브젝트 (object)에 대해서 학습합니다.
물론 강의를 통해서 더욱 자세한 설명을 하지만, 미리 실습해보고 어떤 object인지 개념을 스스로 공부해 보겠습니다.

* The Spark Context
* The Resilient Distributed DataSet or RDD

### Spark Context
* Spark는 강의 시간에 소개한 것과 같이 클러스터에서 computation abstraction을 수행해주는 소프트웨어입니다. 
* Python으로 활용하기 위한 interface를 **pyspark** 라고 합니다.
* **SparkContext** 는 python class로 spark와 사용자 프로그램간의 연동을 책임지는 pyspark의 일부분입니다.

<h3>Spark context 와 RDD 개요</h3>

spark context와 RDD의 정확한 원리를 파악하는 것 보다 기본적 개념을 습득하는 과정입니다. 아래 그림을 통해서 간단하게 설명합니다:

* 아래 그림에서는 3개의 worker node가 존재하며 우리는 spark context를 생성하여, spark context를 통해서 분산된 자원 (코어, 저장소등)을 관리합니다
* 여기서 어떻게 자원이 관리되는지 (어떻게 데이터가 분산 저장되는지)는 spark context에서 알아서 진행하며, 프로그래밍을 하는 사용자 입장에서는 선언만 해주면 됩니다.
* 우리가 활용하는 데이터는 spark에서 RDD라는 포맷으로 관리됩니다
* 예를 들어서 python에서 리스트를 생성해서 RDD로 변환해주면, spark context가 알아서 RDD를 분산 저장합니다.
* 이번 실습에서는 간단하게 RDD (or Resilient Distributed DataSet)는 spark에서 사용하는 데이터 구조로, 다중 클러스터에 저장되는 리스트라고 생각하면 될 것 같습니다.



<p><img alt="" src="https://drive.google.com/uc?id=1mzvJwWSQ5VOC9GZIJdiwFTMd0MWrK2mn" style="height:324px; width:900px" /></p>


설명한 것과 같이 cluster 환경에서 데이터 처리를 대신 맡아줄 spark context를 만들어야합니다. 
Python 환경에서 spark를 사용하기 위해서 꼭 필요한 단계로 생각하면 될 것 같습니다 (초기화).
아래 예제 코드를 실행해 봅니다 (아래 cell을 '클릭' 하면 녹색으로 선택되며, 그 상태에서 shift+enter를 치면 해당 셀이 실행됩니다).
We start by creating a **SparkContext** object named **sc**. In this case we create a spark context that uses 3 *executors*

In [2]:
# pyspark library에서 sparkcontext를 import합니다. (import가 무엇을 하는것인지 모르면 검색을 통해서 알아보면 좋을 것 같습니다)
import findspark
findspark.init()
from pyspark import SparkContext
# SparkContext를 생성하는 명령어
sc = SparkContext(master="local[3]")

# 여기서 master='local[3]'라는 옵션은, 우리가 실제 클러스터 환경에서 실행하는 것이 아니기 때문에 
# local환경에서 '3' 개의 코어를 활용하여 분산 연산을 한다고 선언하는 것입니다.
# 해당 PC의 코어 수가 몇개인지 알아내는 방법은 역시 검색해보죠~

# sc 를 출력시킵니다 (jupyter notebook에서는 셀의 마지막 줄에 표기한 값을 출력합니다, 역시 jupyter 죠)
sc

<SparkContext master=local[3] appName=pyspark-shell>

### 주의!!! 한번에 한개의 sparkcontext 만 생성할 수 있습니다!!
Spark는 기본적으로 한개의 master node로 구성되기 때문에, 일반적으로 (특히 local pc 한대에서 동작하는 우리 입장에서는) 한번에 한개의 spark context만 생성할 수 있습니다. 예를 들어서 위 셀을 다시 실행해보면, 이번에는 에러가 날 것입니다. 이미 sc라는 spark context가 생성되었는데, 다시 생성하라고 명령해서 생기는 에러입니다.

새로 spark context를 생성하기 위해서는 우선 실행된 spark context (sc)를 정지시켜야합니다. 이때 명령어는 
stop( ) method로 아래 셀에 형태로 실행을 합니다. 
실수로 앞에 실행한 sc를 정지하지 않기 위해서 아래 셀은 주석처리 되어 있습니다.

In [42]:
#sc.stop() 

### RDD 생성
위에서 얘기한 것 처럼, 생성한 `RDD`는 **worker nodes**에 분산 저장됩니다. 
이 notebook은 **Master node** 한개에서만 실행되기 때문에 실제 데이터가 있지만, 항상 spark는 클러스터를 위한 체계라는 것을 생각해주세요. 즉 RDD를 master node에 생성 명령한다는 행위는 RDD가 분산 저장된다는 의미이며, 이는 더이상 master node에 이 데이터가 '실제' 저장 되지 않았다는 의미입니다. 실제로는, 분산 저장된 RDD에 대한 주소정보만 갖고 있게 됩니다. 

#### Parallelize 
* RDD를 생성하는 가장 간단한 방법
* 아래 예제코드에서는 `A=sc.parallelize(L)`라는 명령을 통해서, list `L`로 부터 `A`라는 이름의 RDD를 생성하는 과정입니다
* 아래 셀을 실행해 봅니다

In [44]:
A=sc.parallelize(range(3))
A

PythonRDD[1] at RDD at PythonRDD.scala:53

#### Collect

* RDD 여러개의 excutor들 사이에 분산되어 있습니다
* `collect()` 라는 method는 `parallelize()'의 역을(inverse) 수행합니다
* 즉, RDD의 모든 element들을 master node로 수거하여 list를 return하는 method 입니다
* Collect()와 같이 RDD를 다시 master node로 불러와서 실제 값을 확인할 수 있는 method, 함수를 spark에서는 action 이라고 합니다


In [45]:
L=A.collect()
print(type(L))
print(L)

<class 'list'>
[0, 1, 2]


###  `.collect()` 를 사용하면, 병렬 연산에 대한 기회를 소멸시킵니다

프로그래밍을 하다보면, 중간 계산 값을 확인하고싶은 경우가 많이 있을 것입니다. 디버깅 과정에서는 더욱 필요한 과정이겠지만, 최종 프로그램에서는 `.collect()`를 사용하여 RDD를 master node로 불러오는 것은 분산 연산을 포기하는 것과 같습니다 (collect된 데이터는 master node에서만 연산되기 때문에). 

즉 연산을 RDD에 직접 구현하는 것이 필요하며, 이런 RDD에 연산을 하여 다시 다른 RDD로 변환하는 것을 transformation이라고 합니다. 다음 셀에서는 transformation 중에서 가장 기본이 되는 map 함수에 대해서 학습합니다.

### Map
* spark의 map은 python의 map 함수와 결과와 목적은 거의 같습니다
* 즉, python에서 map 함수가 
* 하지만 내부 동작은 전혀 다르며, 이는 수업시간에 배우도록 하겠습니다
* 본 학습에서는 python map 함수와 같다고 생각하면 되며, map 함수는 수행하고자하는 operation 함수를 인자로 받습니다
* 리턴하는 값은 RDD의 값에 map안에 정의한 operation을 각각 수행한 결과를 저장하는 RDD를 리턴합니다
* 여기서 중요한것은 `.map()` 자체는 RDD를 리턴 한다는것입니다!!!! 
* 즉 map()은 transformation이며, master node에 저장되지 않기 때문에 리턴값을 그대로 볼 수는 없죠
* collect()를 통해서 리턴된 값을 확인할 수 있습니다

**Note:** RDD return 예, List return 예

In [46]:
# RDD
print(A.map(lambda x: x*x))
# List
print(A.map(lambda x: x*x).collect())

PythonRDD[2] at RDD at PythonRDD.scala:53
[0, 1, 4]


### Using regular functions instead of lambda functions

* lambda 함수를 활용하면 편한경우가 많지만, 한 줄로 표현이 어려운 경우가 있습니다
* 다음과 같이 일반 함수를 정의해서 사용할 수 있습니다

In [47]:
def square(x):
    return x**2

A.map(square).collect()

[0, 1, 4]

### Excercise 1



---



1. `mapcos`이라는 다음 함수를 만들어보세요: 
  * input: 수(number)로 이루어진 RDD
  * RDD 각 element에 `cos()` (cosine)을 계산해서 list를 리턴

다음 값을 출력하는지 확인하세요
    
```
    [1.0, 0.54030..., -0.41614...]
```


In [48]:
import numpy as np

def mapcos(A):
    
    return A.map(np.cos).collect()  
  # 데이터를 map을 사용해 cos형태로 변환

mapcos(A)

[1.0, 0.5403023058681398, -0.4161468365471424]

### Excercise 2



---



다음 RDD에 대하여 다음 과제를 수행하세요: 

```
stringRDD=sc.parallelize(["Fall semester", "Learning spark basics", "Big data analytics with Spark"])
```
   `mapwords`라는 함수를 작성
   * input: RDD of strings 
   * output: returns a list of words for each string
   
```mapwords(stringRDD)``` 가 다음을 출력하도록 합니다:
    
``` 
[['Fall', 'semester'], ['Learning', 'spark', 'basics'], ['Big', 'data', 'analytics', 'with', 'Spark']]
```

In [78]:
# Excercise 2 답 작성
stringRDD=sc.parallelize(["Fall semester", "Learning spark basics", "Big data analytics with Spark"])
# stringRDD라는 이름의 RDD생성

def mapword(rdd_temp):
    
    return rdd_temp.map(lambda x:x.split()).collect() #split 함수로 리스트안을 공백으로 나눔
  
  # 여기에 코드를 입력하세요.
    

print(mapword(stringRDD))

[['Fall', 'semester'], ['Learning', 'spark', 'basics'], ['Big', 'data', 'analytics', 'with', 'Spark']]


### Reduce

* Reduce는 map과 마찬가지로, python reduce 함수와 같은 기능을 합니다
* python reduce에 대해서 검색해보고 복습해보세요
* 사용 방식은 input으로 function **두개** 의 element를 받는 function을 받으며 **한개**의 output을 리턴합니다
* Reduce는 action 입니다, 즉 `reduce()` 를 실행하면 master node로 값을 실제 리턴합니다
* 마찬가지로 내부적으로는 python reduce와는 다르게 동작하며, 이는 강의를 통해서 배우도록 하겠습니다

가장 간단한 2대1 operation인 합을 구합니다:

In [77]:
A.reduce(lambda x,y: x+y)

3

물론 action이기 때문에 따로 collect는 필요 없이 실제 계산 값이 출력되었을 것입니다

아래 예제는 string list를 RDD로 변환하여 가장 짧은 단어를 찾는 예제입니다:

In [75]:
words=['this','is','the','best','computer','ever']
wordRDD=sc.parallelize(words)
wordRDD.reduce(lambda w,v: w if len(w)<len(v) else v)

'is'

### Exercise 3



---



1. `reduce` command를 활용하여 ```RDD=sc.parallelize([0,2,1])``` RDD에서 가장 큰수를 출력하도록 작성하세요

   Output: ``` 2 ```
   

2. 위 Exercise 2에서 정의한 stringRDD를 `reduce` command 를 활용하여 다음과 같이 하나의 string으로 출력되도록 하세요:

    Output: ``` 'Fall semester Learning spark basics Big data analytics with Spark' ```

In [72]:
# Exercise 3-1 답 작성
RDD = sc.parallelize([0,2,1]) # 0 ,2 ,1 리스트를 가진 RDD생성
def MaxRDD(RDD):
    return RDD.reduce(lambda x,y: x if x > y else y) 
# 리스트의 순서를 누적적으로 하나씩 함수적용, ex) x에 0과 y에 2를 대입하여 조건문으로 하나씩 비교한 후 최댓값 출력
print(MaxRDD(RDD))

2


In [79]:
# Exercise 3-2 답 작성
def CombineString(stringRDD): 
    return stringRDD.reduce(lambda x,y: x+y) 
# stringRDD에 reduce command를 사용하여 이미 나누어진 stirngRDD를 x와 y로 하나씩 대입하여 하나의 문장으로 만듦
print(CombineString(stringRDD))

Fall semesterLearning spark basicsBig data analytics with Spark


### Exercise 4







위 예제에서 작성한 wordRDD에 reduce operation을 취해서 다음 결과가 출력되도록 아래 largerThan 함수를 완성하세요
suppose we want to find the 

* 가장 긴 단어들 중에서 (길이가 같은 단어 존재)
* lexiographic (a,b,c, 순) 순서가 가장 뒤인 단어를 출력하세요

   Output: ``` 'computer' ```


In [69]:
# Exercise 4 답 작성
def largerThan(x,y):
    # 여기에 코드 작성 
    if len(x) > len(y): # wordRDD의 words리스트의 elements들의 길이를 비교하여
        return x # x문자열의 길이가 y문자열의 길이보다 길면 x 반환
    else:
        return y # 그렇지 않으면 y반환

# 결과출력 
wordRDD.reduce(largerThan)

'computer'

### Exercise 5



---



1. 다음 RDD에 대해서 아래 과정을 수행하세요:

    ``` listRDD=sc.parallelize([[3,4],[2,1],[7,9]]) ```
 
     `reduce` command 를 활용하여 리스트안에 수들 중에서 가장 큰 수를 출력하도록 함수를 완성하세요. 출력값은 다음과 같아야 합니다:
     
     Output: ```[9]```
     
     (Note: 출력값이 하나의 수가 아닌 1개 element의 리스트입니다)

In [39]:
# Exercise 5 답안 작성
listRDD=sc.parallelize([[3,4],[2,1],[7,9]])

def LeastNumber(listRDD):
    return listRDD.reduce(lambda x,y: x if x > y else y) 
# 우선 바깥 리스트 3개 중에 가장 큰 수를 뽑아낸다.
  
listRDD2 = sc.parallelize(LeastNumber(listRDD))
# 그런 후 listRDD2라는 RDD를 만들어 그 뽑아낸 리스트를 비교한다.
print(LeastNumber(listRDD2)) # 비교된 리스트중 큰 수를 출력


9


### **Exercise 6**

- - - -
1. `reduce` command를 활용하여 입력받은 수(6개 이상, 10<=N<=20) N의 팩토리얼을 구하는 식을 작성하세요.
    
    **단, 입력받은 수만큼의 RDD를 생성하세요(6개 이상)**
    
   ex)
   
   ```Input ``` : 10 11 13 16 18 20
   
   ```output :  ```
   
   10 의 factorial :  3628800
   
   11 의 factorial :  39916800
   
   13 의 factorial :  6227020800
   
   16 의 factorial :  20922789888000
   
   18 의 factorial :  6402373705728000
   
   20 의 factorial :  2432902008176640000


In [7]:
# Exercise 6 답안 작성
sc.stop()
sc = SparkContext(master="local[*]")

facto = map(int,input().split()) # 공백 문자로 띄어진 입력받을 input함수를 생성 및 facto에 저장

for i in facto: # 처음부터 입력받은 수까지 반복
    rial = []  # 공백 리스트 하나 생성
    for j in range(i, 0,-1):# 처음부터 i의 -1까지 반복
        rial.append(j)
    factoRDD = sc.parallelize(rial) # 입력 받은 수 만큼 RDD생성
        
    print(str(i)+"의 factorial : ",factoRDD.reduce(lambda x,y : x*y)) #action인 reduce로 출력



# 여기에 코드 작성

10 11 13 16 18 20
10의 factorial :  3628800
11의 factorial :  39916800
13의 factorial :  6227020800
16의 factorial :  20922789888000
18의 factorial :  6402373705728000
20의 factorial :  2432902008176640000


### **Exercise 7**

- - - -

HW1의 과제 2번 문제를 RDD를 이용해서 해결하세요. 

**단, value를 직접적으로 sort 하지 말 것(발견시 0점처리) ==> value.sort() X**


```input``` :  value = [46,960,19,182,365,929,568,627,510,355,313,82,742,656,385,181,827,197,809,413] 

```output``` :

              minimum sum =  8206
              maximum sum =  9147 


In [107]:
# Exercise 7 답안 작성

sc.stop()
sc = SparkContext(master="local[*]")
value = [46,960,19,182,365,929,568,627,510,355,313,82,742,656,385,181,827,197,809,413]
valueRDD = sc.parallelize(value) # value 리스트에 대한 valueRDD 생성

maxnum = valueRDD.reduce(lambda x,y : x if x > y else y) #value 리스트 중 reduce를 사용하여 리스트 내 가장 큰 값 구한다.
totalsum=valueRDD.reduce(lambda x,y : x+y) # value 리스트를 reduce command를 사용하여 리스트 내 모든 값을 합한다.
minnum=valueRDD.reduce(lambda x,y : x if x < y else y) # value 리스트 중 reduce command를 사용하여 리스트를 하나하나 비교해 최솟값을 구한다.
print("maximum num : ",maxnum) # 최댓값 출력
print("minimum num : ",minnum) # 최솟값 출력
print("minimum sum = ",totalsum-maxnum) # 전체 합 중에서 최댓값인 960을 뺀다
print("maximum sum = ",totalsum-minnum) # 전체 합 중에서 최솟값인 19를 뺀다.
# 여기에 코드 작성

maximum num :  960
minimum num :  19
minimum sum =  8206
maximum sum =  9147


### **Exercise 8**
#### python의 reduce와 spark의 reduce의 수행시간 비교.

- - - -

```동일한 변수```를 통해서 결과를 도출한 수행시간을 비교한 예제입니다.

```%%time```을 통해 수행시간을 비교한 결과 ```python의 reduce```가 더 빠르게 작동하는 것을 확인할 수 있습니다.

**이유는 무엇일까요?(아직 배우지는 않았지만 추측을 통해서 2 ~ 4 문장 이내로 서술할 것)**

In [0]:
sc.stop()
sc = SparkContext(master="local[*]")

value = np.ones(10**5, dtype = 'i').tolist()

In [0]:
%%time

print("\n ===============pyspark reduce=================")
RDD = sc.parallelize(value)
print("pyspark reduce all sum : ", RDD.reduce(lambda x, y : x + y))


 ===============pyspark reduce=================
pyspark reduce all sum :  100000
CPU times: user 40 ms, sys: 0 ns, total: 40 ms
Wall time: 651 ms


In [0]:
%%time

from functools import reduce
print("\n ===============python reduce=================")
print("python reduce all sum : ", reduce(lambda x, y : x + y, value))


 ===============python reduce=================
python reduce all sum :  100000
CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 25.8 ms


#### Exercise 8 답안 작성

: python의 reduce는 데이터를 처리할 때, 순차적으로 접근하여 순차적으로 처리하지만 pyspark의 reduce는 데이터를 각 부분으로 나누어 처리 한 후 결합하는 형태이다. 이 결합하는 시간이 순차적으로 접근하여 바로 처리하는 python의 reduce보다 소모되기 때문에 더 느리다. 즉, 직렬처리인 python이 병렬처리인 pyspark의 

### 수고하셨습니다